In [2]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [3]:
# Load the data into a Pandas DataFrame
# Ensure the dataset has relevant features before proceeding
file_path = "Resources/crypto_market_data.csv"
df_market_data = pd.read_csv(file_path, index_col="coin_id")

# Display sample data to understand its structure
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [4]:
# Generate summary statistics to check feature distributions
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [5]:
# Visualize the dataset trends
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [6]:
# Standardize the dataset using StandardScaler
scaler = StandardScaler()
df_market_data_scaled = scaler.fit_transform(df_market_data)

In [7]:
# Create a DataFrame with the scaled data
# Maintaining index for reference
df_market_data_scaled = pd.DataFrame(
    df_market_data_scaled, columns=df_market_data.columns, index=df_market_data.index
)
# Copy the crypto names from the original DataFrame
crypto_names = df_market_data.index

# Set the coin_id column as index
df_market_data_scaled.index = crypto_names

# Display the scaled DataFrame
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [8]:
# Determine the optimal k-value using the elbow method
k_values = list(range(1, 12))

In [9]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)  

In [10]:
# Create a DataFrame for plotting the Elbow Curve
df_elbow_data = pd.DataFrame({"k": k_values, "inertia": inertia})

In [11]:
# Plot the Elbow Curve to identify the best k-value
df_elbow_data.hvplot.line(x='k', y='inertia', title='Elbow Curve', xticks=k_values)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for `k` is typically found at the "elbow" of the curve, where inertia starts to decrease at a slower rate. In this case K is around 4 

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [12]:
# Based on the elbow method, selecting k=4 as the optimal cluster count
kmeans_model = KMeans(n_clusters=4, random_state=42)

In [13]:
# Fit the K-Means model using the scaled DataFrame
kmeans_model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=42)

In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
clusters = kmeans_model.predict(df_market_data_scaled)
# Print the resulting array of cluster values.
print(clusters)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [15]:
# Create a copy of the scaled DataFrame
df_market_data_clustered = df_market_data_scaled.copy()

In [16]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_clustered['Cluster'] = clusters

# Display the copy of the scaled DataFrame
df_market_data_clustered.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,2
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,2
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [17]:
# Visualizing clusters based on price percentage changes
df_market_data_clustered.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols=["coin_id", "Cluster"]
)

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id,Cluster)

---

### Optimize Clusters with Principal Component Analysis.

In [18]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [19]:
# Implement PCA to reduce dimensionality while retaining variance
df_market_data_pca = pca.fit_transform(df_market_data_scaled)

# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(
    df_market_data_pca, columns=["PC1", "PC2", "PC3"], index=df_market_data.index
)

# Copy the crypto names from the original scaled DataFrame
df_market_data_pca["coin_id"] = crypto_names

# Set the coin_id column as index
df_market_data_pca.set_index("coin_id", inplace=True)

# Display the scaled PCA DataFrame
df_market_data_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [20]:
# Checking the explained variance to ensure PCA effectiveness
explained_variance = pca.explained_variance_ratio_.sum()
print(f'Total Explained Variance: {explained_variance:.2f}')

Total Explained Variance: 0.90


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 90%, meaning most of the data’s variance is captured by these three components.

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [21]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [22]:
# Finding optimal k-value after PCA transformation
inertia_pca = []
for k in k_values:
    model_pca = KMeans(n_clusters=k, random_state=42)
    model_pca.fit(df_market_data_pca)
    inertia_pca.append(model_pca.inertia_)

df_elbow_pca = pd.DataFrame({"k": k_values, "inertia": inertia_pca})
df_elbow_pca.hvplot.line(x='k', y='inertia', title='Elbow Curve (PCA)', xticks=k_values)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The optimal value for `k` after PCA transformation is typically lower, around k=3 or k=4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Yes, due to reduced feature complexity.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [23]:
# Based on PCA optimization, k=3 seems ideal
kmeans_pca = KMeans(n_clusters=3, random_state=42)
kmeans_pca.fit(df_market_data_pca)

KMeans(n_clusters=3, random_state=42)

In [24]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
pca_clusters = kmeans_pca.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print(pca_clusters)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 0 0 2 2 1
 2 2 2 2]


In [25]:
# Assigning PCA-based clusters
df_market_data_pca_clustered = df_market_data_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_market_data_pca_clustered['Cluster'] = pca_clusters

# Display the dataframe
df_market_data_pca_clustered.head()

,PC1,PC2,PC3,Cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,0
ethereum,-0.458261,0.458466,0.952877,0
tether,-0.433070,-0.168126,-0.641752,2
ripple,-0.471835,-0.222660,-0.479053,2
bitcoin-cash,-1.157800,2.041209,1.859715,0


In [26]:
# Visualizing PCA-based clusters
df_market_data_pca_clustered.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id", "Cluster"]
)


:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2,coin_id,Cluster)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [27]:
# Compute and compare silhouette scores to assess clustering quality
silhouette_original = silhouette_score(df_market_data_scaled, clusters)
silhouette_pca = silhouette_score(df_market_data_pca, pca_clusters)

print(f'Silhouette Score (Original Data): {silhouette_original:.3f}')
print(f'Silhouette Score (PCA Data): {silhouette_pca:.3f}')

Silhouette Score (Original Data): 0.314
Silhouette Score (PCA Data): 0.394


In [28]:
# Compare elbow curves before and after PCA
df_elbow_data.hvplot.line(x='k', y='inertia', label='Original Data') * df_elbow_pca.hvplot.line(x='k', y='inertia', label='PCA Data')

:Overlay
   .Curve.Original_Data :Curve   [k]   (inertia)
   .Curve.PCA_Data      :Curve   [k]   (inertia)

In [29]:
# Compare clustering results before and after PCA
df_market_data_clustered.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='Cluster',
    title='Clusters Using Original Data'
) * df_market_data_pca_clustered.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='Cluster',
    title='Clusters Using PCA Data'
)

:Overlay
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Reducing the number of features using PCA simplifies the clustering process, improves efficiency, and can make clusters more distinct while retaining most of the important variance in the data.